## Deploy HuggingFace pretrained Image-totext model on SageMaker

#### Model used: https://huggingface.co/nlpconnect/vit-gpt2-image-captioning

###### Any Image for this notebook will do the job

##### Upgrade Sagemaker

In [2]:
#pip install "sagemaker>=2.140.0" "transformers==4.26.1" "datasets[s3]==2.10.1" --upgrade

##### Get roles

In [47]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

#### Load the model data we uplodaed to S3

In [40]:
from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://image-captioning-yurui/model.tar.gz",  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.26",                           # Transformers version used
   pytorch_version="1.13",                                # PyTorch version used
   py_version='py39',                                    # Python version used
)



#### Create an inference endpoint

In [43]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge",
)

----!

#### Running inference and sample output

![](https://huggingface.co/datasets/Narsil/image_dummy/raw/main/parrots.png)

In [53]:
# example request: you always need to define "inputs"
data = {
   "inputs": ['https://huggingface.co/datasets/Narsil/image_dummy/raw/main/parrots.png']
}

# request
predictor.predict(data)[0]

'["a pair of parrots standing next to each other"]'

#### Delete endpoint

In [37]:
# delete endpoint
predictor.delete_endpoint()